# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# Verify the quantity of rows
a = 0
b = 0
for row in full_data_rows_list:
    if (row[0] == ''):
        a += 1
    elif (row[0] != ''):
        b += 1

print("Total of rows in list: {} \nTotal of row[0] is null: {} \nTotal of row[0] is not null: {} \nTotal of rows read: {}".format(len(full_data_rows_list), a, b, a+b))

# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print("Number of rows in csv file: ", sum(1 for line in f)-1) # remove the header

Total of rows in list: 8056 
Total of row[0] is null: 1236 
Total of row[0] is not null: 6820 
Total of rows read: 8056
Number of rows in csv file:  6820


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

#### Create tables to run the following queries.  
Remember, with Apache Cassandra you model the database tables on the queries you want to run.  

Create queries to ask the following three questions of the data:  
- ***Query 01:*** Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
- ***Query 02:*** Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
- ***Query 03:*** Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Query 01 - ETL Pipeline Processing

In [8]:
# STEP 01 - ANALYZE the data
# Analyze the data in event_datafile_new.csv to identify the necessary columns for primary key in query 01
file = 'event_datafile_new.csv'
event_df = pd.read_csv(file)

# Analyze the necessary columns for primary key in query 01
event_df.query('sessionId in (3, 5, 8, 9, 10, 15, 19)')

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
1071,The Killers,Alivia,F,1,Terrell,246.80444,free,"Parkersburg-Vienna, WV",3,Read My Mind,4
3812,Kid Rock,Rylan,M,1,George,296.95955,free,"Birmingham-Hoover, AL",15,All Summer Long (Album Version),16
3814,The Killers,Rylan,M,2,George,220.89098,free,"Birmingham-Hoover, AL",15,When You Were Young,16
3817,Trey Songz,Rylan,M,3,George,205.81832,free,"Birmingham-Hoover, AL",15,Can't Help But Wait (Album Version),16
3819,Billy J Kramer & The Dakotas,Rylan,M,4,George,191.32036,free,"Birmingham-Hoover, AL",15,Tennessee Waltz (Live),16
3821,Dashboard Confessional,Rylan,M,5,George,224.54812,free,"Birmingham-Hoover, AL",15,Ghost Of A Good Thing,16
5327,Sugarland,Cecilia,F,0,Owens,247.77098,free,"Atlanta-Sandy Springs-Roswell, GA",5,Just Might (Make Me Believe),6
5328,A Hope For Home,Cecilia,F,1,Owens,388.38812,free,"Atlanta-Sandy Springs-Roswell, GA",5,Absolution: Of Flight and Failure,6
5787,Anne-Lie RydÃÂ©,Wyatt,M,0,Scott,176.90077,free,"Eureka-Arcata-Fortuna, CA",8,SÃÂ¥n't ÃÂr Livet (You Can Have Her) (2002 ...,9
5788,The Smiths,Wyatt,M,1,Scott,242.91220,free,"Eureka-Arcata-Fortuna, CA",8,There Is A Light That Never Goes Out,9


#### Primary Key identify each row because:
 - One userId can have many sessionId's
 - Each sessionId is unique
 - If userId listened to the same song in same sessionId then the itemInSession's are differents
 - The sessionId is the partition key, and item_in_session is the clustering column to make each Primary Key unique
 
 In this case session_id and item_in_session identify each row in this table.

In [9]:
# STEP 02 - CREATE table for query 01
# Query 01: artist, song and length where sessionId = 338 and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS song_session "
query = query + "(session_id int, item_in_session int, song_title text, song_length double, artist_name text, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)           

In [10]:
# STEP 03 - INSERT data into table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_session (session_id, item_in_session, song_title, song_length, artist_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[9], float(line[5]), line[0]))

In [11]:
# STEP 04 - VALIDATION - VERIFY total of rows LOADED
query_count = "select count(*) as tt from song_session"
try:
    rows = session.execute(query_count)
except Exception as e:
    print(e)

for row in rows:
    print (row)

Row(tt=6820)


In [12]:
# STEP 05 - VALIDATION - VERIFY spcific information in table
query = "select * from song_session WHERE session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.song_title, ' - ', row.song_length, ' - ', row.artist_name, ' - ', row.session_id, ' - ', row.item_in_session)

Music Matters (Mark Knight Dub)  -  495.3073  -  Faithless  -  338  -  4


In [13]:
# STEP 06 - VALIDATION - VERIFY spcific information in df origin
event_df = pd.read_csv(file)
columns = ['song', 'length', 'artist', 'sessionId', 'itemInSession']
song_df = event_df[columns]

song_df.query('sessionId == 338 and itemInSession == 4')

,song,length,artist,sessionId,itemInSession
5810,Music Matters (Mark Knight Dub),495.3073,Faithless,338,4


#### Query 02 - ETL Pipeline Processing

In [14]:
# STEP 01 - ANALYZE the data
# Analyze the necessary columns for primary key in query 02
event_df.query('userId == 19 and sessionId == 592')

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
5607,Divinyls,Zachary,M,0,Thomas,227.68281,free,"New York-Newark-Jersey City, NY-NJ-PA",592,I Touch Myself,19
5609,Drowning Pool,Zachary,M,1,Thomas,229.35465,free,"New York-Newark-Jersey City, NY-NJ-PA",592,Reason I'm Alive (Explicit),19
5610,Brujeria,Zachary,M,2,Thomas,143.25506,free,"New York-Newark-Jersey City, NY-NJ-PA",592,Pititis_ Te Invoco (Album Version),19
5611,Joyce Cooling,Zachary,M,3,Thomas,248.11057,free,"New York-Newark-Jersey City, NY-NJ-PA",592,It's Time I Go (Jazz),19
5612,The White Stripes,Zachary,M,4,Thomas,160.31302,free,"New York-Newark-Jersey City, NY-NJ-PA",592,Blue Orchid,19
5613,R.I.O.,Zachary,M,5,Thomas,217.12934,free,"New York-Newark-Jersey City, NY-NJ-PA",592,When The Sun Comes Down,19
5614,Bap,Zachary,M,6,Thomas,277.21098,free,"New York-Newark-Jersey City, NY-NJ-PA",592,FÃÂ¼r Maria (Album Version),19


#### Primary Key identify each row because:
 - One userId can have many sessionId's
 - Each sessionId is unique
 - In sessionId the itemInSession's are differents for each song
 - The userId is the partition key, and from there I will add clustering columns on sessionId and itemInSession
 
 In this case userId, sessionId and itemInSession identify each row in this table.

In [15]:
# STEP 02 - CREATE table for query 02
## Query 2: artist, song (sorted by itemInSession), user (first and last name) where userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS user_session "
query = query + "(user_id int, session_id int, item_in_session int, user_name text, user_last_name text, artist_name text, song_title text, PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [16]:
# STEP 03 - INSERT data into table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_session (user_id, session_id, item_in_session, user_name, user_last_name, artist_name, song_title)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))

In [17]:
# STEP 04 - VALIDATION - VERIFY total of rows LOADED
query_count = "select count(*) as tt from user_session"
try:
    rows = session.execute(query_count)
except Exception as e:
    print(e)

for row in rows:
    print (row)

Row(tt=6820)


In [18]:
# STEP 05 - VALIDATION - VERIFY spcific information in table
query = "select artist_name, song_title, user_name, user_last_name, item_in_session from user_session WHERE user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist_name,' - ', row.song_title,' - ', row.user_name, ' - ', row.user_last_name,' - ', row.item_in_session)

Down To The Bone  -  Keep On Keepin' On  -  Sylvie  -  Cruz  -  0
Three Drives  -  Greece 2000  -  Sylvie  -  Cruz  -  1
Sebastien Tellier  -  Kilometer  -  Sylvie  -  Cruz  -  2
Lonnie Gordon  -  Catch You Baby (Steve Pitron & Max Sanna Radio Edit)  -  Sylvie  -  Cruz  -  3


In [19]:
# STEP 06 - VALIDATION - VERIFY spcific information in df origin
event_df.head()
columns = ['artist', 'song', 'firstName', 'lastName', 'itemInSession', 'userId', 'sessionId']
user_df = event_df[columns]

user_df.query('userId == 10 and sessionId == 182')

,artist,song,firstName,lastName,itemInSession,userId,sessionId
6805,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz,0,10,182
6806,Three Drives,Greece 2000,Sylvie,Cruz,1,10,182
6807,Sebastien Tellier,Kilometer,Sylvie,Cruz,2,10,182
6808,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz,3,10,182


#### Query 03 - ETL Pipeline Processing

In [20]:
# STEP 01 - ANALYZE the data
# Analyze the necessary columns for primary key in query 03
event_df.head()
event_df.query('song == "All Hands Against His Own"')

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
1313,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29
1839,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
5282,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95


#### Primary Key identify each row because:
 - userId is necessary otherwise the select will display only one row. In this dataset there are three rows with song = 'All Hands Against His Own'
 - The song is the partition key, and the userId is the clustering column
 
 In this case song and userId identify each row in this table.

In [21]:
# STEP 02 - CREATE table for query 03
# Query 03: user name (first and last) where song = 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS song_user "
query = query + "(song_title text, user_id int, user_name text, user_last_name text, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [22]:
# STEP 03 - INSERT data into table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_user (song_title, user_id, user_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [23]:
# STEP 04 - VALIDATION - VERIFY total of rows LOADED
query_count = "select count(*) as tt from song_user"
try:
    rows = session.execute(query_count)
except Exception as e:
    print(e)

for row in rows:
    print (row)

Row(tt=6618)


In [24]:
# STEP 05 - VALIDATION - VERIFY spcific information in table
query = "select song_title, user_name, user_last_name from song_user where song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row)

Row(song_title='All Hands Against His Own', user_name='Jacqueline', user_last_name='Lynch')
Row(song_title='All Hands Against His Own', user_name='Tegan', user_last_name='Levine')
Row(song_title='All Hands Against His Own', user_name='Sara', user_last_name='Johnson')


In [25]:
# STEP 06 - VALIDATION - VERIFY spcific information in df origin
event_df.head()
columns = ['artist', 'song', 'firstName', 'lastName', 'itemInSession', 'userId', 'sessionId']
user_df = event_df[columns]

user_df.query('song == "All Hands Against His Own"')

,artist,song,firstName,lastName,itemInSession,userId,sessionId
1313,The Black Keys,All Hands Against His Own,Jacqueline,Lynch,50,29,559
1839,The Black Keys,All Hands Against His Own,Tegan,Levine,25,80,611
5282,The Black Keys,All Hands Against His Own,Sara,Johnson,31,95,152


### Drop the tables before closing out the sessions

In [26]:
## Drop the table before closing out the sessions
query = "drop table song_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()